In [1]:
using MultiResolutionIterators, CorpusLoaders
include("../src/ner.jl")

remove_ner_label_prefix (generic function with 1 method)

In [36]:
using LinearAlgebra, Statistics

In [6]:
dataset = flatten_levels(collect(CorpusLoaders.load(GMB())) , lvls(GMB, :document)) |> full_consolidate

9418-element Array{Array{CorpusLoaders.GMBWord,1},1}:
 [GMBWord("O", "JJ", "Suspected"), GMBWord("O", "JJ", "Islamist"), GMBWord("O", "NNS", "rebels"), GMBWord("O", "VBP", "have"), GMBWord("O", "VBN", "fired"), GMBWord("O", "NN", "mortar"), GMBWord("O", "NNS", "shells"), GMBWord("O", "IN", "at"), GMBWord("O", "DT", "the"), GMBWord("O", "NN", "palace"), GMBWord("O", "VBN", "used"), GMBWord("O", "IN", "by"), GMBWord("Location", "NNP", "Somalia"), GMBWord("O", "POS", "'s"), GMBWord("O", "JJ", "interim"), GMBWord("O", "NNP", "President"), GMBWord("Person", "NNP", "Abdullahi"), GMBWord("Person", "NNP", "Yusuf"), GMBWord("Person", "NNP", "Ahmad"), GMBWord("O", ".", ".")]                    
 [GMBWord("O", "PRP", "It"), GMBWord("O", "VBD", "was"), GMBWord("O", "RB", "not"), GMBWord("O", "RB", "immediately"), GMBWord("O", "JJ", "clear"), GMBWord("O", "IN", "if"), GMBWord("O", "DT", "the"), GMBWord("O", "NN", "president"), GMBWord("O", "VBD", "was"), GMBWord("O", "IN", "in")  …  GMBWord("O", "W

In [8]:
X = [word.(sentence) for sentence in dataset]
Y = [CorpusLoaders.named_entity.(sentence) for sentence in dataset]

9418-element Array{Array{String,1},1}:
 ["O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "Location", "O", "O", "O", "Person", "Person", "Person", "O"]                             
 ["O", "O", "O", "O", "O", "O", "O", "O", "O", "O"  …  "O", "O", "O", "O", "O", "O", "O", "O", "O", "O"]                                                
 ["O", "O", "O", "O", "O", "O", "O", "O", "O", "O"  …  "O", "O", "O", "O", "O", "O", "O", "Location", "Timex", "O"]                                     
 ["O", "O", "O", "O", "O", "O", "O", "O", "O", "O"  …  "O", "O", "O", "O", "O", "O", "O", "O", "O", "O"]                                                
 ["O", "O", "O", "O", "O", "O", "O", "O", "O", "O"  …  "O", "O", "O", "O", "O", "O", "O", "O", "O", "O"]                                                
 ["O", "O", "O", "O", "O", "O", "Person", "Person", "Person", "O"  …  "O", "O", "O", "O", "O", "O", "Location", "Location", "O", "O"]                   
 ["O", "O", "O", "O", "O", "O", "O", "O", "

In [10]:
unique(vcat(Y...))

8-element Array{String,1}:
 "O"             
 "Location"      
 "Person"        
 "Timex"         
 "Organization"  
 "Artifact"      
 "Event"         
 "Natural_Object"

In [11]:
ner = NERTagger()

NERmodel{BiLSTM_CNN_CRF_Model{Conv{2,2,typeof(identity),TrackedArray{…,Array{Float32,4}},TrackedArray{…,Array{Float32,1}}},Array{Float32,2},Flux.Recur{Flux.LSTMCell{TrackedArray{…,Array{Float32,2}},TrackedArray{…,Array{Float32,1}}}},Dense{typeof(identity),TrackedArray{…,Array{Float32,2}},TrackedArray{…,Array{Float32,1}}},CRF{TrackedArray{…,Array{Float32,2}}},Array{Int64,2}}}(BiLSTM_CNN_CRF_Model{Conv{2,2,typeof(identity),TrackedArray{…,Array{Float32,4}},TrackedArray{…,Array{Float32,1}}},Array{Float32,2},Flux.Recur{Flux.LSTMCell{TrackedArray{…,Array{Float32,2}},TrackedArray{…,Array{Float32,1}}}},Dense{typeof(identity),TrackedArray{…,Array{Float32,2}},TrackedArray{…,Array{Float32,1}}},CRF{TrackedArray{…,Array{Float32,2}}},Array{Int64,2}}(["ORG", "O", "MISC", "PER", "PER", "LOC", "ORG", "ORG", "PER", "PER", "MISC", "MISC", "MISC", "ORG", "LOC", "LOC", "LOC"], Dict('w'=>81,'E'=>34,'7'=>22,'Z'=>55,']'=>57,'o'=>73,'B'=>31,'5'=>20,'h'=>66,'i'=>67…), Dict("newdigate"=>164100,"daufuskie"=>25478

In [78]:
function try_outs(ner, x_in, y_in)
    unique_labels = unique(ner.model.labels)
    num_labels = length(unique_labels)
    confusion_matrix = zeros(Int, (num_labels, num_labels))

    for (x_seq, y_seq) in zip(x_in, y_in)
        preds = ner(x_seq)

        for (pred, logit) in zip(preds, y_seq)
            pred == "MISC" && continue

            if(logit == "O")
                confusion_matrix[findfirst(x -> x==pred, unique_labels), findfirst(x -> x=="O", unique_labels)] += 1
            elseif(logit == "Location")
                confusion_matrix[findfirst(x -> x==pred, unique_labels), findfirst(x -> x=="LOC", unique_labels)] += 1
            elseif(logit == "Person")
                confusion_matrix[findfirst(x -> x==pred, unique_labels), findfirst(x -> x=="PER", unique_labels)] += 1
            elseif(logit == "Organization")
                confusion_matrix[findfirst(x -> x==pred, unique_labels), findfirst(x -> x=="ORG", unique_labels)] += 1
            else
                continue
            end
        end
    end

#     print(confusion_matrix)
    s1 = sum(confusion_matrix, dims=2)
    s2 = sum(confusion_matrix, dims=1)
    dg = diag(confusion_matrix)
    s1 = [s1[1:2]..., s1[4:5]...]
    s2 = [s2[1:2]..., s2[4:5]...]
    dg = [dg[1:2]..., dg[4:5]...]

    a = mean(dg ./ s1)
    b = mean(dg ./ s2)

    println("Precision and recall are:", a, " ", b)
    println("F1 is:", (2 * a * b) / (a + b))
end


try_outs (generic function with 1 method)

In [79]:
try_outs(ner, X, Y)

Precision and recall are:0.8227857518580006 0.7660888458451726
F1 is:0.7934257214885667


In [68]:
i = 5
collect(zip(ner(X[i]), X[i], Y[i]))

24-element Array{Tuple{String,String,String},1}:
 ("MISC", "Pakistani", "O")   
 ("O", "officials", "O")      
 ("O", "say", "O")            
 ("O", "unidentified", "O")   
 ("O", "gunmen", "O")         
 ("O", "have", "O")           
 ("O", "killed", "O")         
 ("O", "three", "O")          
 ("O", "people", "O")         
 ("O", ",", "O")              
 ("O", "including", "O")      
 ("O", "a", "O")              
 ("O", "former", "O")         
 ("O", "government", "O")     
 ("O", "minister", "O")       
 ("O", ",", "O")              
 ("O", "in", "O")             
 ("O", "a", "O")              
 ("O", "semi-autonomous", "O")
 ("O", "tribal", "O")         
 ("O", "region", "O")         
 ("O", "bordering", "O")      
 ("LOC", "Afghanistan", "O")  
 ("O", ".", "O")              

In [76]:
i = 95
collect(zip(ner(X[i]), X[i], Y[i]))

13-element Array{Tuple{String,String,String},1}:
 ("O", "The", "O")                  
 ("MISC", "British", "Organization")
 ("O", "Embassy", "Organization")   
 ("O", "has", "O")                  
 ("O", "advised", "O")              
 ("MISC", "British", "Organization")
 ("O", "citizens", "O")             
 ("O", "to", "O")                   
 ("O", "avoid", "O")                
 ("O", "travel", "O")               
 ("O", "to", "O")                   
 ("LOC", "Khartoum", "Organization")
 ("O", ".", "O")                    